In [32]:
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
load_dotenv()

True

In [33]:
# prompt
system_template = "You are a helpful assistant. Answer all your the questions in best of your ability in {language}."

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        MessagesPlaceholder(variable_name="message")
    ]
)

In [34]:
# model
groq_api_key = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

In [35]:
# function to save chat history based on session id
store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [36]:
chain = prompt | model
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="message")

In [38]:
config = {"configurable":{"session_id":"chat_1"}}
response = with_message_history.invoke(
    {
        "message":[HumanMessage(content="Generate 5 sentences containing word action")],
        "language":"Hindi"
    },
    config=config
)
response.content

'Here are 5 sentences containing the word "action":\n\n1. The movie\'s thrilling plot kept us on the edge of our seats, full of suspenseful **action**.\n2. She decided to take **action** and finally apply for that dream job.\n3. The photographer captured the **action** of the basketball game with stunning clarity.\n4. He felt a sense of satisfaction after taking **action** to help his neighbor in need.\n5. The team brainstormed new marketing strategies, hoping to take swift **action** to boost sales. \n\n\nLet me know if you\'d like more examples! \n'

In [39]:
response = with_message_history.invoke(
    {
        "message":[HumanMessage(content="Generate 1 more sentence")],
        "language":"Hindi"
    },
    config=config
)
response.content

"The superhero's swift **action** saved the city from disaster.  \n"